https://habr.com/ru/articles/486358/

In [9]:
import numpy as np
np.random.seed(1)

Эмбединги 

In [10]:
num_word = 2
num_emb = 4
emb = np.random.randn(num_emb,num_word)
emb

array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ],
       [ 1.74481176, -0.7612069 ]])

Обучаемые матрицы query, key, value

In [11]:
Q = np.random.rand(num_emb,num_emb)
K = np.random.rand(num_emb,num_emb)
V = np.random.rand(num_emb,num_emb)

In [28]:
Q

array([[0.20445225, 0.87811744, 0.02738759, 0.67046751],
       [0.4173048 , 0.55868983, 0.14038694, 0.19810149],
       [0.80074457, 0.96826158, 0.31342418, 0.69232262],
       [0.87638915, 0.89460666, 0.08504421, 0.03905478]])

Queries

In [12]:
q = Q@emb
q

array([[ 1.06184527, -1.64066553],
       [ 0.84990467, -1.32864774],
       [ 2.26848963, -2.77713354],
       [ 1.09279384, -1.72148288]])

Keys

In [13]:
k = K@emb
k

array([[ 0.63191683, -1.59301327],
       [ 2.42361169, -2.99061931],
       [ 1.99897634, -1.92860471],
       [ 2.83093419, -2.65395796]])

Values

In [14]:
v = V@emb
v

array([[ 1.22971688, -2.8583937 ],
       [ 1.59996966, -0.87687308],
       [ 0.72202166, -1.58638268],
       [ 2.58600351, -2.39762567]])

Score

In [15]:
from scipy.special import softmax
s = softmax(k@q.T/np.sqrt(num_emb),1)
s

array([[0.15560988, 0.11350688, 0.56329633, 0.16758691],
       [0.03807379, 0.01846971, 0.89885014, 0.04460637],
       [0.078695  , 0.04712803, 0.78643111, 0.08774586],
       [0.03625581, 0.01775316, 0.90382355, 0.04216748]])

Output

In [16]:
z = s@v
z

array([[ 1.21305615, -1.83973965],
       [ 0.84071245, -1.65789512],
       [ 0.96690739, -1.724229  ],
       [ 0.83461433, -1.6541125 ]])

Многоголовое внимание: 2 головы

In [17]:
n_emb = int(num_emb/2)
Q1,Q2 = np.random.rand(n_emb,n_emb),np.random.rand(n_emb,n_emb)
K1,K2 = np.random.rand(n_emb,n_emb),np.random.rand(n_emb,n_emb)
V1,V2 = np.random.rand(n_emb,n_emb),np.random.rand(n_emb,n_emb)

In [27]:
Q1

array([[0.10233443, 0.41405599],
       [0.69440016, 0.41417927]])

In [18]:
num_word = 2
num_emb = 4
emb1,emb2 = emb[:2,:], emb[2:,:]
emb1, emb2


(array([[ 1.62434536, -0.61175641],
        [-0.52817175, -1.07296862]]),
 array([[ 0.86540763, -2.3015387 ],
        [ 1.74481176, -0.7612069 ]]))

Queries, Keys, Values


In [29]:
q1,q2 = Q1@emb1, Q2@emb2
k1,k2 = K1@emb1, K2@emb2
v1,v2 = V1@emb1, V2@emb2

In [20]:
z1 = softmax(k1@q1.T/np.sqrt(n_emb),1)@v1
z2 = softmax(k2@q2.T/np.sqrt(n_emb),1)@v2
z1,z2

(array([[ 0.95390821, -1.16624787],
        [ 0.95517354, -1.16547395]]),
 array([[ 1.37072146, -2.09086501],
        [ 1.39095728, -2.10836932]]))

In [21]:
z = np.vstack([z1,z2])
z

array([[ 0.95390821, -1.16624787],
       [ 0.95517354, -1.16547395],
       [ 1.37072146, -2.09086501],
       [ 1.39095728, -2.10836932]])

In [32]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

# Пример использования позиционного кодирования
d_model = 4  # размерность модели
max_len = 20  # максимальная длина последовательности

pos_encoding = PositionalEncoding(d_model, max_len)

# Пример применения позиционного кодирования к входным данным
input_sequence = torch.randn(1, 10, d_model)  # пример входной последовательности
output_sequence = pos_encoding(input_sequence)

print(output_sequence.shape)  # вывод размерности выходной последовательности

torch.Size([1, 10, 4])


In [33]:
div_term = torch.exp(torch.arange(0, d_model, 2) * -(torch.log(torch.tensor(10000.0)) / d_model))

In [24]:
div_term

tensor([1.0000, 0.0100])

In [25]:
input_seq = torch.tensor(emb).unsqueeze(0)
input_seq = input_seq.permute(0,2,1)
input_seq.shape

torch.Size([1, 2, 4])

In [26]:
output_sequence = pos_encoding(input_seq)
input_seq.squeeze(0).T, output_sequence.squeeze(0).T


(tensor([[ 1.6243, -0.6118],
         [-0.5282, -1.0730],
         [ 0.8654, -2.3015],
         [ 1.7448, -0.7612]], dtype=torch.float64),
 tensor([[ 1.6243,  0.2297],
         [ 0.4718, -0.5327],
         [ 0.8654, -2.2915],
         [ 2.7448,  0.2387]], dtype=torch.float64))